In [1]:
from qiime2 import Artifact
import tempfile
import os
import pandas as pd

In [2]:
working_dir = os.path.abspath('../')

In [3]:
base_refs=['greengenes', 'silva']
for base in base_refs:
    base_path = working_dir+'/output/'+base+'_reference_taxonomy.qza'
    base_classification = Artifact.load(base_path)
    with tempfile.TemporaryDirectory() as temp_dir:
        base_classification.export_data(temp_dir)
        base_taxonomy=pd.read_csv(temp_dir+'/taxonomy.tsv', '\t')
    m2_path = working_dir+'/output/'+base+'_metaxa2_reference_taxonomy.qza'
    m2_classification=Artifact.load(m2_path)
    with tempfile.TemporaryDirectory() as temp_dir:
        m2_classification.export_data(temp_dir)
        m2_taxonomy=pd.read_csv(temp_dir+'/taxonomy.tsv', '\t')
    base_taxonomy = base_taxonomy.sort_values('Feature ID')
    base_taxonomy = base_taxonomy.reset_index()
    base_taxonomy = base_taxonomy.drop(['Consensus','index'],1)
    m2_taxonomy = m2_taxonomy.sort_values('Feature ID')
    m2_taxonomy = m2_taxonomy.reset_index()
    m2_taxonomy = m2_taxonomy.drop(['Consensus','index'],1)
    results = base_taxonomy.compare(m2_taxonomy)
    results.columns = ['base', 'extended']
    results['sequence'] = base_taxonomy['Feature ID']
    with open(base+'_differential_annotations.fasta', 'w') as fasta_file:
        for row in results.itertuples(False, None):
            fasta_file.write('>'+row[0]+'|'+row[1]+'\n'+row[2]+'\n')
    with open(base+'_differential_annotations.tsv', 'w') as tsv:
        tsv.write('sequence\t'+base+'_annotation\textended_annotation\n')
        for row in results.itertuples(False, None):
            tsv.write(row[2]+'\t'+row[0]+'\t'+row[1]+'\n')